# Polychrom Tutorial: Basic Polymer Simulation

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/darinddv/polychrom/blob/master/tutorials/01_basic_polymer_simulation.ipynb)

Welcome to polychrom! This tutorial will guide you through creating your first polymer simulation using the polychrom library.

**What you'll learn:**
- How to install polychrom and its dependencies
- Create a basic polymer simulation
- Apply forces and run the simulation
- Visualize and analyze results

**About polychrom:**
Polychrom is a powerful Python library for simulating polymer dynamics, specifically designed for modeling chromatin and chromosome organization. It's built on OpenMM's high-performance molecular dynamics engine.

## 1. Installation

First, let's install the required dependencies. In Google Colab, we need to install OpenMM and polychrom dependencies.

In [ ]:
# Install conda and OpenMM (required for polychrom)
!pip install -q condacolab
import condacolab
condacolab.install()

# Install OpenMM via conda
!conda install -c omnia openmm -y

# Install polychrom dependencies
!pip install numpy scipy h5py pandas joblib matplotlib

# Clone and install polychrom
!git clone https://github.com/darinddv/polychrom.git
!cd polychrom && pip install -e .

**⚠️ Important:** After running the installation cell above, you need to **restart the runtime** in Google Colab.

Go to `Runtime` → `Restart runtime` and then continue with the cells below.

## 2. Import Libraries

Let's import all the necessary libraries for our polymer simulation.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import sys

# Add polychrom to path if needed
sys.path.append('/content/polychrom')

# Import polychrom modules
import polychrom
from polychrom import forcekits, forces, simulation, starting_conformations
from polychrom.hdf5_format import HDF5Reporter
import polychrom.polymer_analyses

print(f"Successfully imported polychrom version: {polychrom.__version__ if hasattr(polychrom, '__version__') else 'unknown'}")
print("All imports successful!")

## 3. Setting Up the Simulation

Now let's create our first polymer simulation. We'll simulate a polymer chain with 500 monomers in spherical confinement.

In [ ]:
# Simulation parameters
N = 500  # Number of monomers
platform = "CPU"  # Use CPU platform for Colab compatibility
integrator = "variableLangevin"
temperature = 300  # Kelvin
collision_rate = 0.01
timestep = 170  # femtoseconds

print(f"Setting up simulation with {N} monomers")
print(f"Platform: {platform}")
print(f"Temperature: {temperature} K")

## 4. Creating Initial Conformation

We'll start with a cubic polymer conformation and then set up our simulation object.

In [ ]:
# Create initial polymer conformation
polymer = starting_conformations.grow_cubic(N, boxSize=10)

# Create output directory
output_dir = "polymer_simulation"
os.makedirs(output_dir, exist_ok=True)

# Set up reporter for saving trajectory
reporter = HDF5Reporter(folder=output_dir, max_data_length=50, overwrite=True)

# Create simulation object
sim = simulation.Simulation(
    platform=platform,
    integrator=integrator,
    temperature=temperature,
    collision_rate=collision_rate,
    N=N,
    reporters=[reporter]
)

print(f"Initial polymer shape: {polymer.shape}")
print("Simulation object created successfully!")

## 5. Setting Initial Data and Adding Forces

Now we'll set the initial polymer conformation and add the necessary forces.

In [ ]:
# Set initial conformation
sim.set_data(polymer, center=True)

# Add polymer chain forces (bonds, angles, etc.)
sim.add_force(forcekits.polymer_chains(sim))

# Add spherical confinement
sim.add_force(forces.spherical_confinement(sim, density=0.85, k=1.0))

# Add excluded volume interactions
sim.add_force(forces.polynomial_repulsive(sim, trunc=1.5, radiusMult=1.0))

print("Forces added:")
print("- Polymer chain connectivity")
print("- Spherical confinement")
print("- Excluded volume interactions")

## 6. Energy Minimization

Before running the dynamics, let's minimize the energy to remove any bad contacts.

In [ ]:
# Run energy minimization
sim.local_energy_minimization()
print("Energy minimization completed!")

# Get initial energies
eK, eP = sim.get_energy()
print(f"Initial kinetic energy: {eK:.2f}")
print(f"Initial potential energy: {eP:.2f}")

## 7. Running the Simulation

Now let's run the actual molecular dynamics simulation!

In [ ]:
# Run simulation blocks
num_blocks = 10
steps_per_block = 1000

print(f"Running {num_blocks} blocks of {steps_per_block} steps each...")

energies = []
for i in range(num_blocks):
    # Run one block
    sim.do_block(steps_per_block)
    
    # Get current energies
    eK, eP = sim.get_energy()
    energies.append((eK, eP))
    
    if (i + 1) % 2 == 0:
        print(f"Block {i+1}/{num_blocks}: Ek={eK:.2f}, Ep={eP:.2f}")

print("Simulation completed!")

# Save final trajectory
reporter.dump_data()
print("Trajectory saved to HDF5 files")

## 8. Analysis and Visualization

Let's analyze our simulation results and create some visualizations.

In [ ]:
# Plot energy evolution
energies = np.array(energies)
blocks = np.arange(1, num_blocks + 1)

plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(blocks, energies[:, 0], 'o-', label='Kinetic Energy')
plt.plot(blocks, energies[:, 1], 'o-', label='Potential Energy')
plt.xlabel('Block')
plt.ylabel('Energy')
plt.title('Energy Evolution')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.plot(blocks, energies[:, 0] + energies[:, 1], 'o-', color='red')
plt.xlabel('Block')
plt.ylabel('Total Energy')
plt.title('Total Energy')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Get final polymer conformation
final_coords = sim.get_data()

# Calculate radius of gyration
rg = polychrom.polymer_analyses.radius_of_gyration(final_coords)
print(f"Final radius of gyration: {rg:.2f}")

# Visualize the polymer
fig = plt.figure(figsize=(15, 5))

# 3D visualization
ax1 = fig.add_subplot(131, projection='3d')
ax1.plot(final_coords[:, 0], final_coords[:, 1], final_coords[:, 2], 'b-', alpha=0.7, linewidth=1)
ax1.scatter(final_coords[0, 0], final_coords[0, 1], final_coords[0, 2], color='green', s=50, label='Start')
ax1.scatter(final_coords[-1, 0], final_coords[-1, 1], final_coords[-1, 2], color='red', s=50, label='End')
ax1.set_title('3D Polymer Conformation')
ax1.set_xlabel('X')
ax1.set_ylabel('Y')
ax1.set_zlabel('Z')
ax1.legend()

# XY projection
ax2 = fig.add_subplot(132)
ax2.plot(final_coords[:, 0], final_coords[:, 1], 'b-', alpha=0.7, linewidth=1)
ax2.scatter(final_coords[0, 0], final_coords[0, 1], color='green', s=50, label='Start')
ax2.scatter(final_coords[-1, 0], final_coords[-1, 1], color='red', s=50, label='End')
ax2.set_title('XY Projection')
ax2.set_xlabel('X')
ax2.set_ylabel('Y')
ax2.legend()
ax2.grid(True, alpha=0.3)
ax2.set_aspect('equal')

# Distance from center
ax3 = fig.add_subplot(133)
center = np.mean(final_coords, axis=0)
distances = np.linalg.norm(final_coords - center, axis=1)
ax3.plot(distances, 'b-', alpha=0.7)
ax3.set_title('Distance from Center')
ax3.set_xlabel('Monomer Index')
ax3.set_ylabel('Distance')
ax3.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nPolymer statistics:")
print(f"Number of monomers: {len(final_coords)}")
print(f"Radius of gyration: {rg:.2f}")
print(f"End-to-end distance: {np.linalg.norm(final_coords[-1] - final_coords[0]):.2f}")
print(f"Maximum distance from center: {np.max(distances):.2f}")

## 9. Contact Map Analysis

Let's create a contact map to visualize spatial interactions in the polymer.

In [ ]:
# Calculate contact map
def calculate_contact_map(coords, cutoff=2.0):
    """Calculate contact map with given cutoff distance"""
    n = len(coords)
    contact_map = np.zeros((n, n))
    
    for i in range(n):
        for j in range(i+1, n):
            dist = np.linalg.norm(coords[i] - coords[j])
            if dist < cutoff:
                contact_map[i, j] = 1
                contact_map[j, i] = 1
    
    return contact_map

# Create contact map
contact_map = calculate_contact_map(final_coords, cutoff=2.0)

# Visualize contact map
plt.figure(figsize=(10, 8))
plt.imshow(contact_map, cmap='Reds', origin='lower')
plt.colorbar(label='Contact (1=contact, 0=no contact)')
plt.title('Polymer Contact Map')
plt.xlabel('Monomer Index')
plt.ylabel('Monomer Index')

# Add diagonal lines to highlight structure
plt.plot([0, N], [0, N], 'k--', alpha=0.5, linewidth=0.5)

plt.show()

# Calculate contact statistics
total_contacts = np.sum(contact_map) // 2  # Divide by 2 because matrix is symmetric
contact_density = total_contacts / (N * (N-1) / 2)

print(f"Total contacts (within 2.0 units): {total_contacts}")
print(f"Contact density: {contact_density:.4f}")

## 10. Conclusion

Congratulations! You've successfully:

✅ **Set up a polychrom simulation** with a 500-monomer polymer

✅ **Applied realistic forces** including chain connectivity, confinement, and excluded volume

✅ **Run molecular dynamics** simulation and monitored energy evolution

✅ **Analyzed results** with visualization and contact maps

### Next Steps

Now that you understand the basics, you can:

1. **Explore different forces** - Try different confinement geometries or interaction potentials
2. **Longer simulations** - Run more blocks to see equilibration
3. **Different polymers** - Try ring polymers or multi-chain systems
4. **Loop extrusion** - Move on to the loop extrusion tutorial
5. **Real chromatin modeling** - Apply polychrom to biological systems

### Learn More

- 📖 [Full Documentation](https://polychrom.readthedocs.io/)
- 🧬 [Loop Extrusion Tutorial](https://colab.research.google.com/github/darinddv/polychrom/blob/master/tutorials/02_loop_extrusion_simulation.ipynb)
- 📊 [Contact Map Analysis Tutorial](https://colab.research.google.com/github/darinddv/polychrom/blob/master/tutorials/03_contact_map_analysis.ipynb)
- 💻 [GitHub Repository](https://github.com/darinddv/polychrom)

### Questions or Issues?

If you encounter any problems or have questions:
- Check the [documentation](https://polychrom.readthedocs.io/)
- Look at more [examples](https://github.com/darinddv/polychrom/tree/master/examples)
- Open an issue on [GitHub](https://github.com/darinddv/polychrom/issues)